In [42]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import torch

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [39]:
# Initializing pybullet
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)




# Creating robots
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

start = (0.25,0.25)

static = []

static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,-0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [1.25,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
target = static[3]

goal = (1.25, 0.75)

# For milestone 2 only - setting all non-player targets to have no collision with player robot
# Doesn't work when dragging into each other but might still work?
pybullet.setCollisionFilterGroupMask(player, target, 0, 0)

for robot in static:
    pybullet.setCollisionFilterGroupMask(player, robot, 0, 0)

# Setting colors to distinguish robots. BLUE for player, GREEN for obstacle, RED for target
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])
for robot in static:
    pybullet.changeVisualShape(robot, 0, rgbaColor=[0,1,0,1])
pybullet.changeVisualShape(target, 0, rgbaColor=[1,0,0,1])



# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
            cameraEyePosition=[0.75,0.25,3],
            cameraTargetPosition=[0.75,0.25,0],
            cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.1,
            farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)

# Getting position and orientation
print(pybullet.getBasePositionAndOrientation(player))
print(pybullet.getBasePositionAndOrientation(static[0]))
print(pybullet.getBasePositionAndOrientation(static[1]))
print(pybullet.getBasePositionAndOrientation(static[2]))
print(pybullet.getBasePositionAndOrientation(target))


# Start simulation
pybullet.setRealTimeSimulation(1)

((0.25, 0.25, 0.0), (0.0, 0.0, 0.0, 1.0))
((0.75, 0.25, 0.0), (0.0, 0.0, 1.0, 6.123233995736766e-17))
((0.75, 0.75, 0.0), (0.0, 0.0, 1.0, 6.123233995736766e-17))
((0.75, -0.25, 0.0), (0.0, 0.0, 1.0, 6.123233995736766e-17))
((1.25, 0.75, 0.0), (0.0, 0.0, 1.0, 6.123233995736766e-17))


In [43]:
# RRT
samplingArea = [0.25, 1.25, -0.25, 0.75]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.30738845860818487, 0.5058054127636629, 3.5257876380570576]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.30738845860818487, 0.5058054127636629, 3.5257876380570576]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.30738845860818487, 0.5058054127636629, 3.5257876380570576]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[1.0060097991304298, 0.3712747202576826, 0.7870163475832763]
[0. 0. 0.]
[1.0060097991304298, 0.3712747202576826, 0.7870163475832763]
[0. 0. 0.]
[1.0060097991304298, 0.3712747202576826, 0.7870163475832763]
[0. 0. 0.]
[1.0060097991304298, 0.3712747202576826, 0.7870163475832763]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.30738845860818487, 0.505805412763662

[0.3081882908274268, 0.036305856914389034, 5.831508057728971]
[0. 0. 0.]
[0.6064544271093925, 0.30664906514075685, 5.785993701274605]
[0. 0. 0.]
[0.9405912474814083, 0.7430010048821033, 0.961998768729382]
[0. 0. 0.]
[1.0197257431289184, 0.26239299543043526, 1.8062801302924263]
[0. 0. 0.]
[1.25, 0.75, 0]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.77

[0.7267880666159426, 0.031659832397682, 4.1602701624511305]
[0. 0. 0.]
[0.3081882908274268, 0.036305856914389034, 5.831508057728971]
[0. 0. 0.]
[0.6064544271093925, 0.30664906514075685, 5.785993701274605]
[0. 0. 0.]
[0.9405912474814083, 0.7430010048821033, 0.961998768729382]
[0. 0. 0.]
[1.0197257431289184, 0.26239299543043526, 1.8062801302924263]
[0. 0. 0.]
[1.25, 0.75, 0]
[0. 0. 0.]
[0.7681594496054017, 0.7119228287012497, 5.773111196770995]
[0. 0. 0.]
[0.6246581351945759, 0.014513152670132534, 3.6926864034829454]
[0. 0. 0.]
[0.3007021593379424, -0.10440439360856979, 3.2229138770920986]
[0. 0. 0.]
[0.8232693274933491, -0.05253715288763916, 5.419712626302902]
[0. 0. 0.]
[0.9767467543088669, 0.21025385896382942, 2.0639932059391723]
[0. 0. 0.]
[0.9207812685961987, -0.1305397869824826, 1.954825326590505]
[0. 0. 0.]
[0.9207812685961987, -0.1305397869824826, 1.954825326590505]
[0. 0. 0.]
[0.9207812685961987, -0.1305397869824826, 1.954825326590505]
[0. 0. 0.]
[0.9207812685961987, -0.13053978

[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.30738845860818487, 0.5058054127636629, 3.5257876380570576]
[0. 0. 0.]
[0.5628446963165923, -0.1773334460303655, 0.3383704625048422]
[0. 0. 0.]
[1.0060097991304298, 0.3712747202576826, 0.7870163475832763]
[0. 0. 0.]
[1.127276701711411, -0.10562136951417533, 2.1914946187607716]
[0. 0. 0.]
[0.7267880666159426, 0.031659832397682, 4.1602701624511305]
[0. 0. 0.]
[0.3081882908274268, 0.036305856914389034, 5.831508057728971]
[0. 0. 0.]
[0.6064544271093925, 0.30664906514075685, 5.785993701274605]
[0. 0. 0.]
[0.9405912474814083, 0.7430010048

[0.785554349555948, 0.03804882235193985, 0.17828521794499072]
[0. 0. 0.]
[1.0726264814246982, 0.2613818513238729, 1.09759571429714]
[0. 0. 0.]
[0.826694373379514, 0.4631270087773415, 4.262305095483498]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789

...

[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854, 0.7020512838203656, 3.21542261906323]
[0. 0. 0.]
[1.1529708061475854,

[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.18301198531020435, 0.8580776675103329]
[0. 0. 0.]
[0.6914142951743906, -0.183

[0.54157861107001, 0.14866736645038525, 2.3561841445067193]
[0. 0. 0.]
[0.5440813940035228, 0.28077582485074326, 6.025051834528461]
[0. 0. 0.]
[0.7597342362595815, 0.4343132899963068, 0.8010726029760312]
[0. 0. 0.]
[0.4367083402684825, -0.09936615287547801, 1.2216664756019764]
[0. 0. 0.]
[0.5761018586048304, 0.15924076830747835, 2.329795676090894]
[0. 0. 0.]
[0.3865093694836359, 0.049717698291993795, 1.9545883044973114]
[0. 0. 0.]
[1.0666682500779276, 0.420288609925049, 2.749598172414512]
[0. 0. 0.]
[1.196814208046507, 0.12024822690362791, 0.5629582530910526]
[0. 0. 0.]
[0.9620961195575648, -0.032394034689084084, 3.4353600415425087]
[0. 0. 0.]
[0.8210414881958649, 0.08573854161087735, 1.6300310254055606]
[0. 0. 0.]
[0.2700540118633872, 0.14698626649419388, 2.586689640209027]
[0. 0. 0.]
[0.9603233953244126, 0.5227583955484588, 4.2089788626323825]
[0. 0. 0.]
[0.7230008398125178, 0.06440948465389862, 0.5239586626761721]
[0. 0. 0.]
[1.0146569589895935, 0.30335021699555, 1.9915204231680392]

[0.6246581351945759, 0.014513152670132534, 3.6926864034829454]
[0. 0. 0.]
[0.3007021593379424, -0.10440439360856979, 3.2229138770920986]
[0. 0. 0.]
[0.8232693274933491, -0.05253715288763916, 5.419712626302902]
[0. 0. 0.]
[0.9767467543088669, 0.21025385896382942, 2.0639932059391723]
[0. 0. 0.]
[0.9207812685961987, -0.1305397869824826, 1.954825326590505]
[0. 0. 0.]
[0.4365123895446561, -0.13706735944806114, 2.5257443171425003]
[0. 0. 0.]
[0.8819490826542858, 0.1606422429226081, 6.245647128358687]
[0. 0. 0.]
[0.8844288193757872, 0.38825535630016983, 3.2667133540161513]
[0. 0. 0.]
[1.1627695186019822, 0.6593556126637353, 1.8738013829159441]
[0. 0. 0.]
[0.785554349555948, 0.03804882235193985, 0.17828521794499072]
[0. 0. 0.]
[1.0726264814246982, 0.2613818513238729, 1.09759571429714]
[0. 0. 0.]
[0.826694373379514, 0.4631270087773415, 4.262305095483498]
[0. 0. 0.]
[1.1330410943431952, -0.07875710174246497, 5.744306222789916]
[0. 0. 0.]
[0.5762161388378964, 0.5120497548865143, 3.980850344660357

[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052390378201]
[0. 0. 0.]
[0.30990346993334605, 0.04059498620589974, 2.886052

In [44]:
path = np.flip(path, 0)
print(path)


[[0.25       0.25       0.        ]
 [0.44676459 0.41326446 0.16666209]
 [1.25       0.75       0.        ]
 [1.25       0.75       0.        ]]


In [50]:
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [action[0], action[1], 0]
            pybullet.applyExternalForce(player, -1, action_3d, [0,0,0], pybullet.WORLD_FRAME)
        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [53]:
def getCost(states, target):
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        costs.append(cost)
    return costs

In [54]:
H = 5
G = 1000
T = 500
K = 100

def cem(path):
    pybullet.setRealTimeSimulation(0)
    for i in range(len(path) - 1):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1])
            print(initial_actions)
            print(costs)
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions))]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            initial_actions = np.concatenate(sorted_actions[:K], new_actions)
            
        best_action = initial_actions[0][0] # Because no horizon length
        action = [best_action[0], best_action[1], 0]
        pybullet.applyExternalForce(player, -1, action, [0,0,0], pybullet.WORLD_FRAME)
    
            
            


In [55]:
cem(path)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# MPC
horizon_length = 5
opt_iters = 10
samples = 100
top_samples = 10

device = torch.device('cpu')

In [ ]:
def reward(action):
    

In [ ]:
import test_energy
env = test_energy.FuncMinGTEnv(1, 2, )
CEM = mpc.CEM(horizon_length, opt_iters, samples, top_samples,env,device)

In [38]:
pybullet.disconnect()